# Demo: LED control without dashboard
---
**This demo shows two methods for controlling the LEDs which both make use of the Arduino microblaze subsystem in the PYNQ-Z2 base overlay.**

1) Current sources (channels) and switches individually controlled through Python function calls. This method is ideal for testing individual LEDs or demonstrating the system and is used in ***Demo: Basic LED control***. *Note: I could improve performance of this driver by offloading more processing into the microblaze similar to the second driver but it would still be limited by the matrix addressing.*

2) Pulse-width modulation driver which can configure LEDs to *any* pattern and brightness. This is ideal for fast real-time control of the LEDs with an audio input, as it is not limited by matrix addressing and is used in ***Demo: Audio LED control***.

# 1) Basic driver

In [ ]:
from pynq import Overlay
base = Overlay("base.bit")

In [ ]:
from hearlight.driver import ControlLEDs
system = ControlLEDs(base)

In [ ]:
# ALL SWITCHES OPEN
for i in range(10):
    system.switch_control(i, 0)

In [ ]:
# SWITCH OFF ALL DAC CHANNELS
for i in range(10):
    system.dac_channel_control(i, 0, 0)

In [ ]:
# SET SOFTSPAN RANGE FOR ALL DACS
for i in range(10):
    system.dac_config(i, 1)

In [ ]:
# '1' corresponds to 6.25 mA
system.dac_config?

In [ ]:
from time import sleep

# close switches 1 and 3
system.switch_control(1, 1)
system.switch_control(3, 1)

# switch on DAC channels 2 and 9 at 32768 ADC counts
# since it is 16-bit DAC and the reference is 6.25 mA, the channel current for each channel is around 3.125 mA
system.dac_channel_control(2, 1, 32768)
system.dac_channel_control(9, 1, 32768)

sleep(5)

# SWITCH OFF ALL DAC CHANNELS
for i in range(10):
    system.dac_channel_control(i, 0, 0)
# ALL SWITCHES OPEN
for i in range(10):
    system.switch_control(i, 0)

# 2) PWM driver

In [ ]:
from pynq import Overlay
base = Overlay("base.bit")

In [ ]:
from hearlight.audio import audio_led_driver_setup

from pynq import allocate
import numpy as np

In [ ]:
dac_refs = {0 : 3.125,
    1 : 6.25,
    2 : 12.5,
    3 : 25,
    4 : 50,
    5 : 100,
    6 : 200,
    7 : 300}
dac_ref = 1
switch_max_current = 130
device_max_current = 2500

led_driver_program = audio_led_driver_setup(base)
led_driver_program.leds_configure(dac_ref, switch_max_current, device_max_current) # 6.25 mA limit

# updating this array will set the LEDs to the corresponding current counts value
led_counts = allocate(shape=(10,10), dtype=np.uint16)
led_counts.fill(0)
led_driver_program.leds_start(led_counts)

In [ ]:
led_counts[:] = np.random.randint(low=0, high=65536, size=(10,10))
print(led_counts)

In [ ]:
# mess about with LED patterns...
import cv2

# Create a black image
img = np.zeros((10,10,1), np.uint16)

from time import sleep

while(1):
    for i in range(5):
        led_counts[:,:] = cv2.circle(img,(5,5), i+1, 65535, 1)[:,:,0]
        sleep(0.1)
        img[:,:,:] = 0
    for i in range(4,-1,-1):
        led_counts[:,:] = cv2.circle(img,(5,5), i+1, 65535, 1)[:,:,0]
        sleep(0.1)
        img[:,:,:] = 0

In [ ]:
# reprogramme microblaze to stop driver running
audio_led_driver_setup(base)